<a href="https://colab.research.google.com/github/KhaledGhaleb/PatternRecognition/blob/main/Ass_3_Khaled_Fayed_NoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 3**

Get Corel Reduced Data from Github

In [ ]:
import tensorflow as tf
import os
_URL = 'https://raw.githubusercontent.com/KhaledGhaleb/CorelDataSet/main/CORELREduced.zip'
path_to_zip = tf.keras.utils.get_file('COREL-REduced.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'COREL-REduced')

2220032/2215519 [==============================] - 0s 0us/step


# 1- Pretrained VGG16 

Prepare Train and Validation data:

set ratio of 0.5 validation = 20 * 0.5 = 10 validation

Set Image size to 224*224 * 3 to support VGG16


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 128
IMG_SIZE = (224, 224)
COLOR_MODE = 'rgb' #"grayscale"  #  "rgb"
Color_Num = 3
Valid_ratio = 0.5

train_dataset_VGG16 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = "training",
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )
Validation_dataset_VGG16 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = 'validation',
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )

Found 80 files belonging to 4 classes.
Using 40 files for training.
Found 80 files belonging to 4 classes.
Using 40 files for validation.


Normalize data set

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
#reshape_layer = tf.keras.layers.Reshape(3, 4)
train_normalized_ds_VGG16 = train_dataset_VGG16.map(lambda x, y: (normalization_layer(x), y))
validation_normalized_ds_VGG16 = Validation_dataset_VGG16.map(lambda x, y: (normalization_layer(x), y))

Convert normalized dataset to images and labels in nparray 

In [ ]:
train_images_VGG16,train_labels_VGG16  = list(train_normalized_ds_VGG16.as_numpy_iterator())[0]
validate_images_VGG16,validate_labels_VGG16  = list(validation_normalized_ds_VGG16.as_numpy_iterator())[0]

In [ ]:
#tf.keras.layers.experimental.preprocessing.Resizing( 293, 293, interpolation="bilinear", crop_to_aspect_ratio=False)
#train_images_GoogleNet = train_images.resize(293,)

Generate model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
model_VGG16 = VGG16(weights='imagenet') #,include_top=False
#model.summary()

553467904/553467096 [==============================] - 3s 0us/step


Create Feautre Arrays

In [ ]:
TrainFeaturesArr_VGG16 = model_VGG16.predict(train_images_VGG16)
ValidateFeaturesArr_VGG16 = model_VGG16.predict(validate_images_VGG16)

Add KNN layer with K = 3 and P = 1 

In [ ]:
from sklearn import neighbors
knn_VGG16 = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
knn_VGG16.fit(TrainFeaturesArr_VGG16, train_labels_VGG16)
y_pred_VGG16 = knn_VGG16.predict(ValidateFeaturesArr_VGG16)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('VGG16 Accuracy score: ',accuracy_score(y_pred_VGG16, validate_labels_VGG16))
print('VGG16 F1 Score: ',f1_score(y_pred_VGG16, validate_labels_VGG16, average="macro"))

VGG16 Accuracy score:  0.825
VGG16 F1 Score:  0.845711500974659


# 2- Pretrained GoogleNet (InceptionV3)
using InceptionV3 pretrained with imagenet

Prepare Train and Validation data:

set ratio of 0.5 validation = 20 * 0.5 = 10 validation

Set Image size to 299*299 * 3 to support InceptionV3


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 128
IMG_SIZE = (299, 299)
COLOR_MODE = 'rgb' #"grayscale"  #  "rgb"
Color_Num = 3
Valid_ratio = 0.5

train_dataset_InceptionV3 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = "training",
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )
Validation_dataset_InceptionV3 = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = 'validation',
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )

Found 80 files belonging to 4 classes.
Using 40 files for training.
Found 80 files belonging to 4 classes.
Using 40 files for validation.


Normalize data set

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
#reshape_layer = tf.keras.layers.Reshape(3, 4)
train_normalized_ds_InceptionV3 = train_dataset_InceptionV3.map(lambda x, y: (normalization_layer(x), y))
validation_normalized_ds_InceptionV3 = Validation_dataset_InceptionV3.map(lambda x, y: (normalization_layer(x), y))

Convert normalized dataset to images and labels in nparray 

In [ ]:
train_images_InceptionV3,train_labels_InceptionV3  = list(train_normalized_ds_InceptionV3.as_numpy_iterator())[0]
validate_images_InceptionV3,validate_labels_InceptionV3  = list(validation_normalized_ds_InceptionV3.as_numpy_iterator())[0]

Generate model

In [ ]:
from tensorflow.keras.applications.inception_v3  import InceptionV3
model_InceptionV3 = InceptionV3(weights='imagenet') #,include_top=False
#model.summary()

96116736/96112376 [==============================] - 1s 0us/step


Create Feautre Arrays

In [ ]:
TrainFeaturesArr_InceptionV3 = model_InceptionV3.predict(train_images_InceptionV3)
ValidateFeaturesArr_InceptionV3 = model_InceptionV3.predict(validate_images_InceptionV3)

Add KNN layer with K = 3 and P = 1 

In [ ]:
from sklearn import neighbors
knn_InceptionV3 = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
knn_InceptionV3.fit(TrainFeaturesArr_InceptionV3, train_labels_InceptionV3)
y_pred_InceptionV3 = knn_InceptionV3.predict(ValidateFeaturesArr_InceptionV3)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('InceptionV3 Accuracy score: ',accuracy_score(y_pred_InceptionV3, validate_labels_InceptionV3))
print('InceptionV3 F1 Score: ',f1_score(y_pred_InceptionV3, validate_labels_InceptionV3, average="macro"))

InceptionV3 Accuracy score:  0.975
InceptionV3 F1 Score:  0.9891304347826086


# 3- Use VGG16 in a transfer learning mode

Create VGG16 Model and Freeze this layer

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
model_VGG16_Transfer =  VGG16(weights='imagenet',include_top=False)

#freeze the transfer model.
model_VGG16_Transfer.trainable = False

model_VGG16_Transfer.summary()


58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

*Create* New Model

In [ ]:
import keras
from keras import layers
Transfer_model = keras.Sequential(
    [
        keras.Input(shape=((240,240,3))),
        model_VGG16_Transfer,
        #VGG16(weights='imagenet',include_top=False),
        layers.GlobalAveragePooling2D(),
        #layers.Dense(1024,activation='relu'),
        #layers.Dense(512,activation='relu'),
        layers.Dense(128,activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4, activation="softmax"),
    ]
)

Transfer_model.summary()
for i, layer in enumerate(Transfer_model.layers):
   print(i, layer.name)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 14,780,868
Trainable params: 66,180
Non-trainable params: 14,714,688
_________________________________________________________________
0 module_wrapper
1 global_average_pooling2d
2 dense
3 dropout
4 dense_1


In [ ]:
Transfer_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Transfer_model.fit(x= train_images_VGG16, y= train_labels_VGG16,
                epochs=25,
                batch_size=10,
                shuffle=True,
                validation_data=(validate_images_VGG16,validate_labels_VGG16))

Epoch 1/25
4/4 [==============================] - 15s 314ms/step - loss: 1.6619 - accuracy: 0.1267 - val_loss: 1.5562 - val_accuracy: 0.2750
Epoch 2/25
4/4 [==============================] - 0s 117ms/step - loss: 1.5102 - accuracy: 0.2933 - val_loss: 1.3750 - val_accuracy: 0.3500
Epoch 3/25
4/4 [==============================] - 0s 119ms/step - loss: 1.3757 - accuracy: 0.4067 - val_loss: 1.1765 - val_accuracy: 0.5250
Epoch 4/25
4/4 [==============================] - 1s 156ms/step - loss: 1.2933 - accuracy: 0.3933 - val_loss: 1.0506 - val_accuracy: 0.8500
Epoch 5/25
4/4 [==============================] - 0s 119ms/step - loss: 1.1571 - accuracy: 0.4133 - val_loss: 0.9366 - val_accuracy: 0.9500
Epoch 6/25
4/4 [==============================] - 1s 156ms/step - loss: 0.8622 - accuracy: 0.7467 - val_loss: 0.8688 - val_accuracy: 0.8750
Epoch 7/25
4/4 [==============================] - 0s 118ms/step - loss: 0.8523 - accuracy: 0.8233 - val_loss: 0.8073 - val_accuracy: 0.8750
Epoch 8/25
4/4 [===

Fine Tune

In [ ]:
# Unfreeze the base model
model_VGG16_Transfer.trainable = True
Transfer_model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
                       loss="categorical_crossentropy", metrics=["accuracy"])

Transfer_model.fit(x= train_images_VGG16, y= train_labels_VGG16,
                epochs=3,
                batch_size=10,
                shuffle=True,
                validation_data=(validate_images_VGG16,validate_labels_VGG16))

Epoch 1/3
4/4 [==============================] - 5s 371ms/step - loss: 0.1767 - accuracy: 0.9733 - val_loss: 0.1634 - val_accuracy: 0.9250
Epoch 2/3
4/4 [==============================] - 1s 219ms/step - loss: 0.1496 - accuracy: 0.9533 - val_loss: 0.1969 - val_accuracy: 0.9000
Epoch 3/3
4/4 [==============================] - 1s 253ms/step - loss: 0.0484 - accuracy: 1.0000 - val_loss: 0.1571 - val_accuracy: 0.9000


Scores



In [ ]:
scores = Transfer_model.evaluate(validate_images_VGG16,validate_labels_VGG16, verbose=1) 
print("Transfer VGG16 Accuracy: ", scores[1])

2/2 [==============================] - 0s 50ms/step - loss: 0.1571 - accuracy: 0.9000
Transfer VGG16 Accuracy:  0.8999999761581421


# 4- DCT2 traditional features and the KNN classifier

Get DCT2 Features

In [ ]:
from scipy.fftpack import  dct
DCT2_TrainFeaturesArr = []
for image in train_images_VGG16:
  DCT_out = dct(image, 2)
  DCT_out_1 = DCT_out[:10,:10] 
  DCT_out_2 = DCT_out_1.reshape((300))
  DCT2_TrainFeaturesArr.append(DCT_out_2)

DCT2_ValidateFeaturesArr = []
for image in validate_images_VGG16:
  DCT_out = dct(image, 2)
  DCT_out_1 = DCT_out[:10,:10] 
  DCT_out_2 = DCT_out_1.reshape((300))
  DCT2_ValidateFeaturesArr.append(DCT_out_2)

Add KNN layer with K = 3 and P = 1 

In [ ]:
from sklearn import neighbors
knn_DCT2 = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
knn_DCT2.fit(DCT2_TrainFeaturesArr, train_labels_VGG16)
y_pred_DCT2 = knn_DCT2.predict(DCT2_ValidateFeaturesArr)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('DCT2 KNN Accuracy score: ',accuracy_score(y_pred_DCT2, validate_labels_VGG16))
print('DCT2 KNN F1 Score: ',f1_score(y_pred_DCT2, validate_labels_VGG16, average="macro"))

DCT2 KNN Accuracy score:  0.55
DCT2 KNN F1 Score:  0.5553030303030303


# 5- Train a CNN Model

Prepare Train and Validation data:

set ratio of 0.5 validation = 20 * 0.5 = 10 validation

Set Image size to 100*100 * 3


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 128
IMG_SIZE = (100, 100)
COLOR_MODE = 'rgb' #"grayscale"  #  "rgb"
Color_Num = 3
Valid_ratio = 0.5

train_dataset_CNN = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = "training",
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )
Validation_dataset_CNN = image_dataset_from_directory(PATH,
                                             seed = 1234,                                           
                                             color_mode = COLOR_MODE, 
                                             validation_split = Valid_ratio,
                                             subset = 'validation',
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             label_mode ='categorical',
                                             )

Found 80 files belonging to 4 classes.
Using 40 files for training.
Found 80 files belonging to 4 classes.
Using 40 files for validation.


Normalize data set

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
#reshape_layer = tf.keras.layers.Reshape(3, 4)
train_normalized_ds_CNN = train_dataset_CNN.map(lambda x, y: (normalization_layer(x), y))
validation_normalized_ds_CNN = Validation_dataset_CNN.map(lambda x, y: (normalization_layer(x), y))

Convert normalized dataset to images and labels in nparray 

In [ ]:
train_images_CNN,train_labels_CNN  = list(train_normalized_ds_CNN.as_numpy_iterator())[0]
validate_images_CNN,validate_labels_CNN  = list(validation_normalized_ds_CNN.as_numpy_iterator())[0]

In [ ]:
import keras
from keras import layers
from keras import regularizers
from keras.regularizers import l1,l2
CNN_model = keras.Sequential(
    [
        keras.Input(shape=((IMG_SIZE[0],IMG_SIZE[1],Color_Num))),
        layers.Conv2D(32, kernel_size=(3, 3), strides=2, padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(4, activation="softmax"),
    ]
)

CNN_model.summary()

NameError: ignored

Compile and Fit CNN

In [ ]:
CNN_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

CNN_model.fit(x= train_images_CNN, y= train_labels_CNN,
                epochs=50,
                batch_size=10,
                shuffle=True,
                validation_data=(validate_images_CNN,validate_labels_CNN))

Epoch 1/50
4/4 [==============================] - 2s 116ms/step - loss: 1.3961 - accuracy: 0.2167 - val_loss: 1.3920 - val_accuracy: 0.1750
Epoch 2/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3493 - accuracy: 0.3600 - val_loss: 1.3916 - val_accuracy: 0.1750
Epoch 3/50
4/4 [==============================] - 0s 13ms/step - loss: 1.2739 - accuracy: 0.3100 - val_loss: 1.3755 - val_accuracy: 0.3000
Epoch 4/50
4/4 [==============================] - 0s 15ms/step - loss: 1.1659 - accuracy: 0.4433 - val_loss: 1.2845 - val_accuracy: 0.3750
Epoch 5/50
4/4 [==============================] - 0s 15ms/step - loss: 1.0173 - accuracy: 0.6567 - val_loss: 1.1280 - val_accuracy: 0.4000
Epoch 6/50
4/4 [==============================] - 0s 13ms/step - loss: 0.8305 - accuracy: 0.7833 - val_loss: 0.9519 - val_accuracy: 0.5000
Epoch 7/50
4/4 [==============================] - 0s 15ms/step - loss: 0.7726 - accuracy: 0.6967 - val_loss: 0.8517 - val_accuracy: 0.6000
Epoch 8/50
4/4 [==========

In [ ]:
scores = CNN_model.evaluate(validate_images_CNN,validate_labels_CNN, verbose=1) 
print("CNN Accuracy: ", scores[1])

2/2 [==============================] - 0s 7ms/step - loss: 1.1452 - accuracy: 0.7000
CNN Accuracy:  0.699999988079071


# Conclusion

#Accuracy:

Pretrained VGG16 give accuracy of 0.825 on validation images.

Pretrained GoogleNet (InceptionV3) give accuracy of 0.975 on validation images

Use VGG16 as Transfer model give accuracy of 0.9 on validation data after fine tuning.

DCT2 traditional with KNN give accuracy of  0.55 on validation data, which mean we need to choose other tradional algorithm to extract feature.

CNN Model give accuracy of 0.7 on validation images

#Timing:

DCT2,  Pretrained models (VGG16 and GoogleNet) have about no time to extract features and KNN is fast algorithm. 

Transfer Modeling is fast when freeze the weights, but fine tuning take some time.

CNN Model is the consume a little bit time.

# Final
Using transfer modeling with fine tuning enhanced VGG16 but GoogleNet (InceptionV3) give better accuracy

In [ ]:
#featuresArr = []
#for x in train_images:
#  features = model.predict(x)
#  features_reduce =  features.squeeze()
#  featuresArr.append(features_reduce)

In [ ]:
#from tensorflow.keras.applications.vgg16 import decode_predictions

#urllib.request.urlretrieve("https://user-images.githubusercontent.com/26264000/81228345-f167f600-8fbb-11ea-8722-d25dcda78a0c.jpg", "sample.jpg")
#img_path = 'sample.jpg'
#img = image.load_img(img_path, target_size=(224, 224))
#x = image.img_to_array(img)
#x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

#features = model.predict(x)
#print('Predicted:', decode_predictions(features, top=3)[0])